### Pipeline optimization 
* Objective:
 * Build a stack of numpy arrays depicting the cost distance away from biogas sources
 
* Requirements - Packages
 * NumPy, Pandas, Scikit-Image

* Requirements - Data:
 * CSV of biogas sources, including coordinates and biogas production potential (MMBtu)
 * Cost surface for building pipelines
 
* Output: 
 * Stacked NumPy array
 
 https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#create-least-cost-path

In [1]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal
from matplotlib import pyplot as plt

In [2]:
#Read in biogase sources (example: Duplin Co)
dfBG =  pd.read_excel('../data/DuplinCountySwineFarmEconomics.xlsx',
                         sheet_name='Duplin County Swine Farm Master').iloc[:,[11,12,16]]

In [65]:
#Read in cost surface
ds = gdal.Open('../data/MIT Cost Surface Duplin Co/MIT Cost Surface Duplin.tif')
#Get raster attributes
llx, x_size, x_angle, lly, y_angle, y_size = ds.GetGeoTransform()
#Extract Band1 as the cost array
arrCost = np.array(ds.GetRasterBand(1).ReadAsArray())
#Set zero costs to high costs
arrCost[arrCost == 0] = 10000
#Print info
print("Lower left coordinate = ({0:.8f},{1:.8f})".format(llx,lly))
print("Pixel size is {0:.8f}(x), {1:.8f}(y)".format(x_size,y_size))

Lower left coordinate = (-78.40182301,35.28640434)
Pixel size is 0.00580288(x), -0.00581646(y)


In [21]:
#Create the MCP object from the cost surface
print("Creating the graph from the cost raster")
lc_graph = graph.MCP_Geometric(arrCost,sampling=(x_size,y_size))

Creating the graph from the cost raster


In [78]:
#Iterate through all records and create a cost distance raster
cd_arrays = []
for i,r in dfBG.iterrows():
    print(".",end='')
    x = r[1]
    y = r[0]

    #Get array index from lat long
    xOffset = int(round((x - llx)/x_size))
    yOffset = int(round((y - lly)/y_size))

    #Compute cost distances away
    cd_array = lc_graph.find_costs(starts=([(yOffset, xOffset)]))[0]
    cd_arrays.append(cd_array)

................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [79]:
#Stack all the arrays
arrStack =np.stack(cd_arrays)
np.save('../data/DuplinStack.npy',arrStack)